# DSCI 100 Final Project | Predicting High-Activity Players on a Minecraft Research Server

## Introduction

### Relevant Background Information
The research group in UBC Computer Science operates a Minecraft server that collects behavioural data from players as they move, explore, and interact within the virtual environment. This data helps researchers study navigation, decision-making, and learning in complex digital spaces. However, running such a server requires careful planning: the team must ensure they have enough server capacity, software licenses, and recruitment resources to support various levels of player activity. Because recruitment is limited and resource-intensive, the research team needs to understand which players are likely to generate the most meaningful gameplay data. To guide this planning, the team lead, Frank Wood, presented our data science class with three questions about predicting player behaviour. Our project focuses on Question 2, which aims to identify the kinds of players who contribute large amounts of data so that recruitment efforts can be more effectively targeted.

### Research Question
The guiding question of this project is Frank Wood's question #2:
We would like to know which types of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts.

Response Variable (Dependent):
Total session time or number of sessions per player, calculated from the sessions.csv file. This measures the amount of data a player contributes, as players with longer or more frequent sessions generate more data.

Explanatory Variables (Independent):
experience, subscribe, gender, age

How This Helps Address the Question:
By linking the survey data with objective session logs, we can identify which types of players based on experience, age, gender, and subscription status contribute the most data. This analysis will allow us to predict and target players who are more likely to engage deeply with the game and guide recruiting efforts toward high engagement players.

### Dataset Description
To answer question #2, two datasets were provided by the research group: Players.csv + sessions.csv

#### Dataset: players.csv
The dataset contains survey data of players, including self-reported play hours, subscription status, hashed email addresses, names, genders, and ages. 196 observations 9 variables

Purpose: Captures demographic and playtime information about different players.

| Variable         | Type        | Description                | Missing Values | Notes                                                           |
| ---------------- | ----------- | -------------------------- | -------------- | --------------------------------------------------------------- |
| experience       | Categorical | Self-reported experience   | 0              | May not be truthfully reported                                  |
| subscribe        | Boolean     | Subscription status        | 0              | Binary (true/false)                                             |
| hashedEmail      | String      | Player ID                  | 0              | Used to link with other CSVs; unique and useful for measurement |
| played_hours     | Float       | Self-reported hours played | 0              | May not be truthfully reported                                  |
| name             | String      | Player surname             | 0              | Some names are non-unique                                       |
| gender           | Categorical | Player gender              | 0              | Distribution unknown                                            |
| age              | Int         | Player age                 | 0              | Potential outliers present                                      |
| individualid     | Float       | Account ID                 | 196            | Useless                                                         |
| organizationName | Float       | Organization name          | 196            | Useless                                                         |

Further Notes:

1. Consistent discrepancies exist between self-reported and actual playtime (potential dishonesty).
2. Some younger players report “pro” or “veteran” status, which may skew data.
3. Subscription status appears independent of experience.
 
#### Dataset: sessions.csv
The dataset contains session data from Minecraft server logs, including hashed emails, start and end times, and two additional columns that may contain corrupted timestamps.
1535 observations
5 variables

**Purpose:** Tracks player sessions and frequency to measure in-game activity.

| Variable            | Type   | Description                               | Missing Values | Notes                                            |
| ------------------- | ------ | ----------------------------------------- | -------------- | ------------------------------------------------ |
| hashedEmail         | String | Player identifier (linked to players.csv) | 0              | Used to merge datasets                           |
| start_time          | String | Start time of play session                | 0              | Requires datetime conversion                     |
| end_time            | String | End time of play session                  | 2              | Some missing values prevent duration calculation |
| original_start_time | Float  | Corrupted timestamp                       | 0              | Useless                                          |
| original_end_time   | Float  | Corrupted timestamp                       | 2              | Useless                                          |

**Further Notes:**

1. Some sessions are missing end times, making duration calculations impossible.
2. A single player's hashedEmail may have multiple sessions.
3. Some sessions appear to be logged incorrectly.



## Methods + Results

In [3]:
import pandas as pd
import altair as alt

## Loading the Data

In [11]:
players = pd.read_csv("https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz")
sessions = pd.read_csv("https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB")

In [12]:
players.head(10) #loading the first 10 rows of the dataset #1 to see characteristics

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
5,Amateur,True,f58aad5996a435f16b0284a3b267f973f9af99e7a89bee...,0.0,Adrian,Female,17,NaN,NaN
6,Regular,True,8e594b8953193b26f498db95a508b03c6fe1c24bb5251d...,0.0,Luna,Female,19,NaN,NaN
7,Amateur,False,1d2371d8a35c8831034b25bda8764539ab7db0f6393869...,0.0,Emerson,Male,21,NaN,NaN
8,Amateur,True,8b71f4d66a38389b7528bb38ba6eb71157733df7d17403...,0.1,Natalie,Male,17,NaN,NaN
9,Veteran,True,bbe2d83de678f519c4b3daa7265e683b4fe2d814077f90...,0.0,Nyla,Female,22,NaN,NaN


In [13]:
sesh.head(10) #loading the first 10 rows of the dataset #2 to see characteristics 

,hashedemail,start_time,end_time,original_start_time,original_end_time,time,start_hr_of_day
0,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,2024-06-30 18:12:00,2024-06-30 18:24:00,1.719770e+12,1.719770e+12,720.0,18
1,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,2024-06-17 23:33:00,2024-06-17 23:46:00,1.718670e+12,1.718670e+12,780.0,23
2,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,2024-07-25 17:34:00,2024-07-25 17:57:00,1.721930e+12,1.721930e+12,1380.0,17
3,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,2024-07-25 03:22:00,2024-07-25 03:58:00,1.721880e+12,1.721880e+12,2160.0,3
4,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,2024-05-25 16:01:00,2024-05-25 16:12:00,1.716650e+12,1.716650e+12,660.0,16
5,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,2024-06-23 15:08:00,2024-06-23 17:10:00,1.719160e+12,1.719160e+12,7320.0,15
6,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,2024-04-15 07:12:00,2024-04-15 07:21:00,1.713170e+12,1.713170e+12,540.0,7
7,ad6390295640af1ed0e45ffc58a53b2d9074b0eea694b1...,2024-09-21 02:13:00,2024-09-21 02:30:00,1.726880e+12,1.726890e+12,1020.0,2
8,96e190b0bf3923cd8d349eee467c09d1130af143335779...,2024-06-21 02:31:00,2024-06-21 02:49:00,1.718940e+12,1.718940e+12,1080.0,2
9,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,2024-05-16 05:13:00,2024-05-16 05:52:00,1.715840e+12,1.715840e+12,2340.0,5


## Wrangling + Cleaning the Datasets

### Data Wrangling Plan ####

1. Clean each dataset individually
2. Merge the datasets according to hashedEmail
3. Convert start time and end time to datetime format and compute session lengths
4. Group sessions by hashedEmail to calculate the total playtime per player


In [29]:
# **players.csv cleaning**
# standardizing column names
players.columns = (
    players.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=False)
)

#getting rid of emtpy columns
players = players.dropna(axis=1, how="all")

# dropping useless columns
players = players.drop(columns=["name", "individualid", "organizationname", "played_hours"],
                       errors="ignore")

# checking to make sure hashedemail exists and its unique and good to use
players = players.dropna(subset=["hashedemail"])
players["hashedemail"] = players["hashedemail"].astype(str).str.strip()
players = players.sort_values("hashedemail").drop_duplicates(subset="hashedemail")

# making sure all are numerical values
players["age"] = pd.to_numeric(players["age"], errors="coerce")

# changing to boolean values to make it easier for later (not as cateogrical values)
players["subscribe"] = (
    players["subscribe"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map({"true": True, "false": False})
)

#keeping only important columns
players = players[["hashedemail", "experience", "subscribe", "gender", "age"]]

#check to see if clean or not
players.head(10)


,hashedemail,experience,subscribe,gender,age
61,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,Regular,True,Male,20
60,055d1a2002f6308a95b03b8d17af28f43e9f91ca8b1e1f...,Veteran,True,Male,17
170,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,Pro,False,Male,21
153,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,Beginner,True,Male,17
40,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,Regular,True,Male,17
62,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,Pro,True,Male,17
145,11006065e9412650e99eea4a4aaaf0399bc338006f85e8...,Veteran,False,Male,19
24,119f01b9877fc5ea0073d05602a353b91c4b48e4cf02f4...,Amateur,True,Female,21
131,11bf6125c4264b3a8f3bffa57b33bd598e2ea1ecd6331a...,Amateur,False,Female,23
43,126ec350278422ba60ad575d230c19522fd568094b7c0f...,Veteran,True,Non-binary,21


In [30]:

# standardizing column names
sessions.columns = (
    sessions.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=False)
)

# dropping empty columns
sessions = sessions.dropna(axis=1, how="all")

# dropping unuseable times
sessions = sessions.drop(
    columns=["original_start_time", "original_end_time"],
    errors="ignore"
)

# doing same thing as before (checking hashedemail)
sessions = sessions.dropna(subset=["hashedemail"])
sessions["hashedemail"] = sessions["hashedemail"].astype(str).str.strip()

# remove duplicates
sessions = (
    sessions
    .sort_values(["hashedemail", "start_time", "end_time"])
    .drop_duplicates()
)

# Remove rows with missing end times
sessions = sessions[sessions["end_time"].notna()]

sessions["start_time"] = pd.to_datetime(
    sessions["start_time"], errors="coerce", dayfirst=True
)
sessions["end_time"] = pd.to_datetime(
    sessions["end_time"], errors="coerce", dayfirst=True
)

# get rid of rows where time conversion did not work
sessions = sessions.dropna(subset=["start_time", "end_time"])

# computing the session lengths in minutes (standardizing time unit)
sessions["session_length"] = (
    sessions["end_time"] - sessions["start_time"]
).dt.total_seconds() / 60

#only keeping sessions that exist in the players.csv file (if they do not its not proven to be)
sessions = sessions[sessions["hashedemail"].isin(players["hashedemail"])]

sessions.head()


,hashedemail,start_time,end_time,session_length
361,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,2024-05-22 23:12:00,2024-05-23 00:13:00,61.0
1458,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,2024-05-23 00:22:00,2024-05-23 01:07:00,45.0
976,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,2024-06-28 04:28:00,2024-06-28 04:58:00,30.0
664,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,2024-09-19 21:01:00,2024-09-19 21:12:00,11.0
1399,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,2024-08-24 02:41:00,2024-08-24 02:46:00,5.0


In [94]:
#merging cleaned datasets #1 + #2 + checking to make sure they merged properly
merged = sessions.merge(players, on="hashedemail", how="inner")

merged.head(10)


,hashedemail,start_time,end_time,session_length,experience,subscribe,gender,age
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,2024-05-22 23:12:00,2024-05-23 00:13:00,61.0,Regular,True,Male,20
1,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,2024-05-23 00:22:00,2024-05-23 01:07:00,45.0,Regular,True,Male,20
2,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,2024-06-28 04:28:00,2024-06-28 04:58:00,30.0,Pro,False,Male,21
3,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,2024-09-19 21:01:00,2024-09-19 21:12:00,11.0,Beginner,True,Male,17
4,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,2024-08-24 02:41:00,2024-08-24 02:46:00,5.0,Regular,True,Male,17
5,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,2024-08-24 03:15:00,2024-08-24 03:48:00,33.0,Regular,True,Male,17
6,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,2024-08-27 19:18:00,2024-08-27 19:52:00,34.0,Regular,True,Male,17
7,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,2024-08-29 04:08:00,2024-08-29 04:52:00,44.0,Regular,True,Male,17
8,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,2024-08-30 03:40:00,2024-08-30 04:04:00,24.0,Regular,True,Male,17
9,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,2024-08-30 04:05:00,2024-08-30 04:36:00,31.0,Regular,True,Male,17


In [96]:
# starting from your cleaned merged dataframe
# merged should already contain:
# hashedemail, session_length, experience, subscribe, gender, age

# 1. make a per player session summary
session_summary = (
    merged
    .groupby("hashedemail", as_index=False)
    .agg(
        total_playtime_min=("session_length", "sum"),
        session_count=("session_length", "size"),
        avg_session_length_min=("session_length", "mean")
    )
)

# 2. keep only the predictor columns once per player
player_info = (
    merged[["hashedemail", "experience", "subscribe", "gender", "age"]]
    .drop_duplicates(subset="hashedemail")
)

# 3. merge predictors with the session summary
player_summary = (
    player_info
    .merge(session_summary, on="hashedemail", how="inner")
)

player_summary.head(10)


,hashedemail,experience,subscribe,gender,age,total_playtime_min,session_count,avg_session_length_min
0,0088b5e134c3f0498a18c7ea6b8d77b4b0ff1636fc9335...,Regular,True,Male,20,106.0,2,53.000000
1,060aca80f8cfbf1c91553a72f4d5ec8034764b05ab59fe...,Pro,False,Male,21,30.0,1,30.000000
2,0ce7bfa910d47fc91f21a7b3acd8f33bde6db57912ce02...,Beginner,True,Male,17,11.0,1,11.000000
3,0d4d71be33e2bc7266ee4983002bd930f69d304288a866...,Regular,True,Male,17,418.0,13,32.153846
4,0d70dd9cac34d646c810b1846fe6a85b9e288a76f5dcab...,Pro,True,Male,17,70.0,2,35.000000
5,11006065e9412650e99eea4a4aaaf0399bc338006f85e8...,Veteran,False,Male,19,10.0,1,10.000000
6,119f01b9877fc5ea0073d05602a353b91c4b48e4cf02f4...,Amateur,True,Female,21,50.0,1,50.000000
7,18936844e06b6c7871dce06384e2d142dd86756941641e...,Amateur,True,Male,14,1217.0,41,29.682927
8,1a2b92f18f36b0b59b41d648d10a9b8b20a2adff550ddb...,Beginner,False,Female,37,18.0,1,18.000000
9,1d2371d8a35c8831034b25bda8764539ab7db0f6393869...,Amateur,False,Male,21,5.0,1,5.000000


In [97]:
# checking to put them in the proper order of ascending to see that to players and etc.


player_summary.sort_values("total_playtime_min", ascending = False).head(20)

,hashedemail,experience,subscribe,gender,age,total_playtime_min,session_count,avg_session_length_min
84,bfce39c89d6549f2bb94d8064d3ce69dc3d7e72b38f431...,Regular,True,Male,17,14671.0,219,66.990868
77,b622593d2ef8b337dc554acb307d04a88114f2bf453b18...,Regular,True,Non-binary,20,14132.0,94,150.340426
73,ad6390295640af1ed0e45ffc58a53b2d9074b0eea694b1...,Regular,True,Female,19,11827.0,147,80.455782
117,f8f5477f5a2e53616ae37421b1c660b971192bd8ff77e3...,Amateur,True,Female,16,9947.0,159,62.559748
122,fd6563a4e0f6f4273580e5fedbd8dda64990447aea5a33...,Amateur,True,Male,23,4795.0,310,15.467742
19,36d9cbb4c6bc0c1a6911436d2da0d09ec625e43e6552f5...,Amateur,True,Male,17,3873.0,130,29.792308
69,a175d4741dc84e6baf77901f6e8e0a06f54809a34e6b52...,Amateur,True,Female,17,3232.0,37,87.351351
14,24d4892c2c3ce11d3e54c3bf31ee218901cac7a2f56491...,Amateur,True,Male,22,2366.0,79,29.949367
114,f6daba428a5e19a3d47574858c13550499be23603422e6...,Pro,True,Male,9,2019.0,27,74.777778
65,9782452b8e41d1f140374eec34c84f464df6ed12e2777e...,Beginner,True,Male,24,1581.0,18,87.833333


## Summary of Merged Dataset

To prepare for our analysis, we summarized key variables in the merged dataset to understand the distribution of gameplay activity and player characteristics. We focused on total playtime, session count and average session length as the variables to quantify the amount of gameplay data each player contributes. We also examined the categorical vairables including experience, gender, and subscription status to understand the composition of different player types in the dataset. 

Moreover, the summary statistics we generated show the distribution of different player types. This EDA provide initial insights into variation in engagement levels across the player population and informs the analysis of which types of players contribute the most data. 

In [40]:
# statistical summaries for evaluation of other values 
player_summary[["total_playtime_min", "session_count", "avg_session_length_min"]].describe()

,total_playtime_min,session_count,avg_session_length_min
count,125.000000,125.000000,125.000000
mean,623.728000,12.264000,32.511372
std,2325.466909,41.312566,32.456914
min,5.000000,1.000000,5.000000
25%,11.000000,1.000000,10.000000
50%,32.000000,1.000000,18.500000
75%,106.000000,2.000000,43.000000
max,14671.000000,310.000000,150.340426


In [44]:
#distribution of experience groups, gender counts and proportion fo subscribers vs non
player_summary["experience"].value_counts()


experience
Amateur     42
Veteran     27
Regular     23
Beginner    22
Pro         11
Name: count, dtype: int64

In [45]:
player_summary["gender"].value_counts()

gender
Male                 83
Female               24
Non-binary            6
Prefer not to say     6
Two-Spirited          4
Agender               1
Other                 1
Name: count, dtype: int64

In [46]:
player_summary["subscribe"].value_counts()

subscribe
True     93
False    32
Name: count, dtype: int64

In [43]:
# age summary of evaluation for larger age population
player_summary["age"].describe()

count    125.000000
mean      21.800000
std       11.511565
min        8.000000
25%       17.000000
50%       19.000000
75%       23.000000
max       99.000000
Name: age, dtype: float64

## Visualizations

In [54]:
# Visualization 1: Distribution of Total PLaytime (minutes)

chart1 = alt.Chart(player_summary).mark_bar().encode(
    x=alt.X('total_playtime_min:Q', 
            bin=alt.Bin(maxbins=30), 
            title='Total Playtime (minutes)'),
    y=alt.Y('count():Q', 
            title='Number of Players'),
    tooltip=['count()']
).properties(
    title='Figure 1: Distribution of Total Playtime per Player',
    width=500,
    height=300
)

#Visualization 2: sessions vs average ssession leangth: colored by experience

chart2 = alt.Chart(player_summary).mark_circle(size=70, opacity=0.7).encode(
    x=alt.X('session_count:Q', title='Number of Sessions'),
    y=alt.Y('avg_session_length_min:Q', title='Average Session Length (minutes)'),
    color=alt.Color('experience:N', title='Experience'),
    tooltip=['hashedemail:N', 'session_count:Q', 'avg_session_length_min:Q', 'experience:N']
).properties(
    title='Figure 2: Sessions vs Average Session Length by Experience',
    width=500,
    height=300
)
#Visualization 3: average total playtime by experience

chart3 = alt.Chart(player_summary).mark_bar().encode(
    x=alt.X('experience:N', title='Experience Level'),
    y=alt.Y('mean(total_playtime_min):Q', title='Average Total Playtime (minutes)'),
    color='experience:N',
    tooltip=['experience:N', 'mean(total_playtime_min):Q']
).properties(
    title='Figure 3: Average Total Playtime by Experience Level',
    width=500,
    height=300
)

#Visualization 4: total playtime by subscription status

chart4 = alt.Chart(player_summary).mark_boxplot().encode(
    x=alt.X('subscribe:N', title='Subscription Status'),
    y=alt.Y('total_playtime_min:Q', title='Total Playtime (minutes)'),
    color='subscribe:N',
    tooltip=['subscribe:N', 'total_playtime_min:Q']
).properties(
    title='Figure 4: Distribution of Total Playtime by Subscription Status',
    width=500,
    height=300
)

In [55]:
# grid of all the charts shown

(chart1 & chart2) & (chart3 & chart4).resolve_scale(color='independent')


alt.VConcatChart(...)

Explanation of Visualizations: 

Chart1: The histogram shows how much gameplay time each player contributed. The distribution is strongly right skewed with most players contrinbuting little and a small numebr contrinbuting very large amounts, this higlights the importance of identifying which player characteristics predict these high-engagement individuals

Chart2: The scatterplot compares how many sessions players completed and how long their sessions tend to be, with points colored by experience level. Different experience groups show distinct patterns in session behaviour which helps to further reveal whether certain player types such as pro or veteran tend to engage more deeply with the server

Chart3: This bar chart displays the mean total playtime for each self reported expereince group. Some experience categoires clearly contribute more gameplay time than others, because experience is a key player characteristic. THis visualization directly supports the analysis of which player types generates the most data. 

Chart4: The boxplot compares total playtime between players who subscribed to the newsletter and those who did not. The difference in medians and spread reveals whether subscribers tend to engage more. Since subscription may indicate commitment or interest, this helps assess whether subscribers are a high value player type for recruitment. 

## Data Analysis

Method: KNN Neighbors Regression 

Assumptions: 
- players with similar predictor values will have similar total playtime
- distance in the feature space meaningfully represents similarity
- features are properly scaled so no variable dominates the distance metric
- the training data reflects real behaviour pattenrs and contains representative neighbor examples

Limitations: 
- KNN struggles when categorical vairabels dominates the features space, which may reduce meaningful distance comparisons
- sensitive to feature scaling; improper prprocessing can distort neighborhoods
- performance decreases for highly skewed outcomes
- predictions can be unstabel fro players who have few close neighbors
- KNN becomes less interpretable than linear regression because preidcitions are example rather than equation
- can be wrongly predicted if people self reported values wrong such as experience


Model Comparison: 
- KNN evaluted with Root Mean Squared Prediction Error (RMSPE)- standard regression metric
- cross validation is used to select the optimal k, balancinv over and underfitting
- compared to linear regression which was in our proposals, KNN can model non linear relationships but sacrifices interpretability
- the best model is selected based on lowest cross-validated RMSPE

Plan: 
1. cleaned + wrangled
2. Split data into 80% training and 20% testing sets
3. Use 5 fold cross validation to tune neighbors
4. evaluate final performance on the held out test set using RMSPE and visualization to see if good or not

In [72]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error

feature_cols = ["experience", "subscribe", "gender", "age"]
X = player_summary[feature_cols].copy()
y = player_summary["total_playtime_min"].copy()

# drop missing values
mask = X.notna().all(axis=1)
X = X[mask]
y = y[mask]

# doing the train test split with the plan of: 80% and 20%
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=1,
    shuffle=True
)

# creating the preprocesser and pipeline
cat_cols = ["experience", "subscribe", "gender"]
num_cols = ["age"]

preprocess = ColumnTransformer(
    transformers=[
        (
            "cat",
            OneHotEncoder(drop="first", handle_unknown="ignore"),
            cat_cols
        ),
        ("num", StandardScaler(), num_cols),
    ]
)

knn = KNeighborsRegressor()

pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("knn", knn),
])

# tunig with 5 fold cross validation
cv = 5
n_train = X_train.shape[0]

candidate_ks = list(range(1, 101))

# trying to find the max k
max_k = int((n_train * (cv - 1)) / cv)
valid_ks = [k for k in candidate_ks if k <= max_k]

print("n_train:", n_train)
print("max_k per fold:", max_k)
print("Valid k values being tested:", valid_ks)

param_grid = {
    "knn__n_neighbors": valid_ks
}

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=cv,
    scoring="neg_mean_squared_error",  # we will convert to RMSPE

    
    n_jobs=-1
)

grid.fit(X_train, y_train)

# best k and RMSPE
best_k = grid.best_params_["knn__n_neighbors"]
best_cv_mse = -grid.best_score_
best_cv_rmspe = best_cv_mse ** 0.5

print("Best k (CV):", best_k)
print("Cross-validated RMSPE:", best_cv_rmspe)

y_pred = grid.predict(X_test)

test_mse = mean_squared_error(y_test, y_pred)
test_rmspe = test_mse ** 0.5

print("Test RMSPE:", test_rmspe)


n_train: 100
max_k per fold: 80
Valid k values being tested: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80]
Best k (CV): 34
Cross-validated RMSPE: 2495.4518779361697
Test RMSPE: 1138.0587936114123


In [104]:
import pandas as pd
import altair as alt

#prediction vs true values
results_df = pd.DataFrame({
    "actual_total_playtime_min": y_test.values,
    "predicted_total_playtime_min": y_pred
})

# scatter plot
scatter = alt.Chart(results_df).mark_circle(size=60, opacity=0.7).encode(
    x=alt.X("actual_total_playtime_min:Q", title="Actual Total Playtime (minutes)"),
    y=alt.Y("predicted_total_playtime_min:Q", title="Predicted Total Playtime (minutes)"),
    tooltip=["actual_total_playtime_min", "predicted_total_playtime_min"]
).properties(
    title="Figure 5: Predicted vs Actual Total Playtime",
    width=500,
    height=300
)

line = alt.Chart(pd.DataFrame({
    "x": [
        results_df["actual_total_playtime_min"].min(),
        results_df["actual_total_playtime_min"].max()
    ]
})).mark_line(color="red").encode(
    x="x:Q",
    y="x:Q"
)

scatter + line


alt.LayerChart(...)

In [105]:
cv_results = pd.DataFrame({
    "k": grid.cv_results_["param_knn__n_neighbors"].astype(int),
    "mean_cv_mse": -grid.cv_results_["mean_test_score"]
})

cv_results["mean_cv_rmspe"] = cv_results["mean_cv_mse"] ** 0.5

chart_k = alt.Chart(cv_results).mark_line(point=True).encode(
    x=alt.X("k:Q", title="Number of Neighbours (k)"),
    y=alt.Y("mean_cv_rmspe:Q", title="Cross-validated RMSPE"),
    tooltip=["k", "mean_cv_rmspe"]
).properties(
    title="Figure 6: Cross-validated RMSPE Across K Values",
    width=500,
    height=300
)

chart_k


alt.Chart(...)

In [98]:
# Get predictions for all players (not just test set)
player_summary["predicted_playtime"] = grid.predict(X)

In [107]:
# predicted playtime vs experience
chart_exp = alt.Chart(player_summary).mark_boxplot().encode(
    x=alt.X("experience:N", title="Experience Level"),
    y=alt.Y("predicted_playtime:Q", title="Predicted Total Playtime (min)"),
    color="experience:N"
).properties(
    title="Figure 7: Predicted Playtime by Experience Level",
    width=400, height=300
)
chart_exp


alt.Chart(...)

In [108]:
# predicted playtime vs subscription status

chart_sub = alt.Chart(player_summary).mark_boxplot().encode(
    x=alt.X("subscribe:N", title="Subscribed?"),
    y=alt.Y("predicted_playtime:Q"),
    color="subscribe:N"
).properties(
    title="Figure 8: Predicted Playtime by Subscription Status",
    width=300, height=300
)
chart_sub


alt.Chart(...)

In [109]:
chart_gender = alt.Chart(player_summary).mark_boxplot().encode(
    x="gender:N",
    y="predicted_playtime:Q",
    color="gender:N"
).properties(
    title="Figure 9: Predicted Playtime by Gender",
    width=300, height=300
)
chart_gender


alt.Chart(...)

In [110]:
chart_age = alt.Chart(player_summary).mark_circle(size=40, opacity=0.6).encode(
    x="age:Q",
    y="predicted_playtime:Q",
    tooltip=["hashedemail", "age", "predicted_playtime"]
).properties(
    title="Figure 10: Predicted Playtime vs Age",
    width=400, height=300
)
chart_age


alt.Chart(...)

## Results

No predictors are specific, besides subscription, prediction should be that: higher predicted playtime conincides with higher age, experience level and subscription  status. But only the preidction that matches is the subscribers are higher, while gender is completely irrelevant as well as age and experience level. This may mainly be due to the self reported experience level which many be under reported or over reported, and with age and gender being irrelevant as it may not specifically decide if they have been playing as much. 


Best k (CV): 34
Cross-validated RMSPE: 2495.4518779361697
Test RMSPE: 1138.0587936114123

- High since such large deviations and range of values from the playtime of ~15000 minutes to ~0 minutes.
- Predicted high RMSPE valid